In [1]:
!$SPARK_HOME

/bin/bash: /usr/local/spark: Is a directory


In [1]:
import findspark
findspark.init("/usr/local/spark")

from pyspark.sql import SparkSession 
import pyspark.sql.functions as F 

from delta import *

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .master("spark://namenode:7077")\

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3dc1d966-5e9e-4930-b36e-9480c5adf311;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 272ms :: artifacts dl 16ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|ev

In [2]:
business=spark.read.parquet('hdfs://namenode:9000/project_data/data/business/')

business.write.format("delta").mode("overwrite").save("/temp/business")

df = spark.read.format("delta").load("/temp/business")

df.show(5)

In [3]:
reviews=spark.read.parquet('hdfs://namenode:9000/project_data/data/reviews/')

In [4]:
reviews.take(1)

[Row(review_id='6fObpwIggOQR1oDapyQzyg', user_id='lN-1uUHeV_QyFbczw8ErlA', business_id='grpNey31cTGKrhmQQaNGdA', stars='5.0', useful=0, funny=0, cool=0, text='Had a wonderful, authentic basque dinner Friday night.  Service was great and friendly and the bar was fun. Thanks Gavin!  We will definitely be back!Chris and Cathy (Turlock CA)', date='2020-06-28 19:45:32')]

In [5]:
business.createOrReplaceTempView("business")
reviews.createOrReplaceTempView("reviews")
combined_table = spark.sql("SELECT r.review_id, b.business_id, r.text, r.date, b.categories, r.stars FROM reviews AS r LEFT JOIN business AS b ON b.business_id = r.business_id ")


In [6]:
combined_table.show(5)

+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|           review_id|         business_id|                text|               date|          categories|stars|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|6fObpwIggOQR1oDap...|grpNey31cTGKrhmQQ...|Had a wonderful, ...|2020-06-28 19:45:32|Mexican, Spanish,...|  5.0|
|UgtrUhfuEgUdPay75...|Fay6yoOC6iitEt3QL...|Yeah it was defin...|2020-06-07 18:13:33|Italian, Pizza, F...|  4.0|
|MmLxg9oLQmPpcPNqI...|wQq0QBaYXa1KLNw_J...|One of the last s...|2018-08-17 18:51:47|Restaurants, Arts...|  4.0|
|awQEOCuJ9fL12h7iq...|xgJMQq0uVY4KB9Efn...|Great experience ...|2020-06-15 14:31:14|Donuts, Coffee & ...|  5.0|
|R-Jgku-ExdyGh5qOk...|ps1Dduk_Oqrr_O2no...|Our family recent...|2020-02-03 14:10:57|Restaurants, Wine...|  5.0|
+--------------------+--------------------+--------------------+-------------------+--------------------

In [7]:
combined_table.count()

6990280

In [8]:
restaurant_reviews = combined_table.where(F.col('categories').contains("Restaurants"))

In [9]:
restaurant_reviews.count()

4724468

In [10]:
restaurant_reviews.show(5)

+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|           review_id|         business_id|                text|               date|          categories|stars|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|6fObpwIggOQR1oDap...|grpNey31cTGKrhmQQ...|Had a wonderful, ...|2020-06-28 19:45:32|Mexican, Spanish,...|  5.0|
|UgtrUhfuEgUdPay75...|Fay6yoOC6iitEt3QL...|Yeah it was defin...|2020-06-07 18:13:33|Italian, Pizza, F...|  4.0|
|MmLxg9oLQmPpcPNqI...|wQq0QBaYXa1KLNw_J...|One of the last s...|2018-08-17 18:51:47|Restaurants, Arts...|  4.0|
|R-Jgku-ExdyGh5qOk...|ps1Dduk_Oqrr_O2no...|Our family recent...|2020-02-03 14:10:57|Restaurants, Wine...|  5.0|
|QUzo-EaJleFtHZSQ0...|tEHgpkQT50Z-M0Xaf...|Walt did it again...|2020-02-27 19:40:08|Steakhouses, Rest...|  5.0|
+--------------------+--------------------+--------------------+-------------------+--------------------

In [12]:
restaurant_reviews.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/temp/filtered_reviews")